In [19]:
from torchsummary import summary
from torch import nn
import torch
from collections import OrderedDict

### 4.1 定义模型类

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 压平输入数据
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

In [18]:
summary(model,input_size=(1, 28, 28),batch_size=64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [64, 784]               0
            Linear-2                  [64, 512]         401,920
              ReLU-3                  [64, 512]               0
            Linear-4                  [64, 512]         262,656
              ReLU-5                  [64, 512]               0
            Linear-6                   [64, 10]           5,130
              ReLU-7                   [64, 10]               0
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 1.39
Params size (MB): 2.55
Estimated Total Size (MB): 4.14
----------------------------------------------------------------


### 4.2 定义模型容器

#### Sequential

In [42]:
X = torch.randn((1,784))

In [47]:
torch.manual_seed(42)
model1 = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

In [49]:
torch.manual_seed(42)
layer2=OrderedDict([
    ('linear1',nn.Linear(28*28, 512)),
    ('relu1',nn.ReLU()),
    ('linear2',nn.Linear(512, 512)),
    ('relu2',nn.ReLU()),
    ('linear3',nn.Linear(512, 10)),
    ('relu3',nn.ReLU())
])
model2 = nn.Sequential(layer2)

In [52]:
torch.manual_seed(42)
model3=nn.Sequential()
model3.add_module('linear1',nn.Linear(28*28, 512))
model3.add_module('relu1',nn.ReLU())
model3.add_module('linear2',nn.Linear(512, 512))
model3.add_module('relu2',nn.ReLU())
model3.add_module('linear3',nn.Linear(512, 10))
model3.add_module('relu3',nn.ReLU())

In [54]:
model1(X), model2(X), model3(X)

(tensor([[0.0000, 0.0066, 0.0020, 0.0000, 0.0000, 0.0000, 0.0131, 0.0932, 0.0000,
          0.0439]], grad_fn=<ReluBackward0>),
 tensor([[0.0000, 0.0066, 0.0020, 0.0000, 0.0000, 0.0000, 0.0131, 0.0932, 0.0000,
          0.0439]], grad_fn=<ReluBackward0>),
 tensor([[0.0000, 0.0066, 0.0020, 0.0000, 0.0000, 0.0000, 0.0131, 0.0932, 0.0000,
          0.0439]], grad_fn=<ReluBackward0>))

#### ModuleDict

In [63]:
class MyModuleDict(nn.Module):
    def __init__(self):
        super(MyModuleDict, self).__init__()
        self.choices = nn.ModuleDict({
                'conv': nn.Conv2d(10, 10, 3),
                'pool': nn.MaxPool2d(3)
        })
        self.activations = nn.ModuleDict([
                ['lrelu', nn.LeakyReLU()],
                ['prelu', nn.PReLU()]
        ])

    def forward(self, x, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x

In [64]:
X=  torch.randn((1, 10, 32, 32))
model_dict4= MyModuleDict()
output = model_dict4.forward(X,'conv','lrelu')

#### ModuleList

In [67]:
class MyModuleList(nn.Module):
    def __init__(self):
        super(MyModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(10)])

    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
        return x

In [68]:
X = torch.randn((1, 10))
model5=MyModuleList()
output=model5(X)

#### 不用容器的写法

In [84]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear1=nn.Linear(28*28, 512)
        self.linear2=nn.Linear(512, 512)
        self.linear3=nn.Linear(512, 10)
        self.relu1=nn.ReLU()
        self.relu2=nn.ReLU()
        self.relu3=nn.ReLU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        return x
    
model = NeuralNetwork()

In [85]:
summary(model,input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
              ReLU-5                  [-1, 512]               0
            Linear-6                   [-1, 10]           5,130
              ReLU-7                   [-1, 10]               0
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.55
Estimated Total Size (MB): 2.58
----------------------------------------------------------------
